## **Sistema de substituição de indices previstos pela ML**

In [1]:
import openpyxl

# Dicionário de substituição
substituir_indices = {
    "ACG": "IPCA",
    "MME": "IPCA",
    "AIC": "IPCA",
    "AME": "IPCA",
    "INS": "IPCA",
    "MOE": "IPCA",
    "EQN": "IPAME"
}

# Abrir o arquivo original mantendo formatação
file_path = "C:\\Users\\FUDO\\Documents\\CSFR\\doc-entrada\\csfr_preenchido.xlsx"
wb = openpyxl.load_workbook(file_path)
ws = wb.active  # Aba principal

# Ler cabeçalhos
header = [cell.value for cell in ws[1]]

# Localizar colunas de índice, peso e fatorK
colunas_indices = [(i, header[i]) for i in range(len(header)) if header[i] and header[i].startswith("Indice")]
colunas_pesos = [(i, header[i]) for i in range(len(header)) if header[i] and header[i].startswith("Peso")]
colunas_fator = [(i, header[i]) for i in range(len(header)) if header[i] and header[i].startswith("FatorK")]

# Mapear células com cor (preenchidas)
color_map = {}
for row in ws.iter_rows(min_row=2):
    for cell in row:
        if cell.fill.patternType == "solid":
            color_map[(cell.row, cell.column)] = True

# Adicionar cabeçalho da nova coluna de fórmula sintética
col_formula = ws.max_column + 1
ws.cell(row=1, column=col_formula).value = "Formula sintetica"

# Processar linha por linha
formulas_set = set()
for row in ws.iter_rows(min_row=2):
    valores = []
    pesos_dict = {}
    indices_dict = {}
    cor_existe = False

    for idx, (i, _) in enumerate(colunas_indices):
        cel_idx = (row[0].row, i + 1)
        if cel_idx not in color_map:
            continue

        cor_existe = True
        val = row[i].value
        if val in substituir_indices:
            val = substituir_indices[val]
            row[i].value = val

        peso = row[colunas_pesos[idx][0]].value or 0.0
        try:
            peso = float(peso)
        except:
            peso = 0.0

        if val:
            if val not in pesos_dict:
                pesos_dict[val] = peso
                indices_dict[val] = idx
            else:
                pesos_dict[val] += peso

    if not cor_existe:
        ws.cell(row=row[0].row, column=col_formula).value = "-"
        continue

    soma_total = sum(pesos_dict.values())
    if soma_total > 0:
        arredondados = [round(v / soma_total, 6) for v in pesos_dict.values()]
        resto = 1.0 - sum(arredondados)
        if resto != 0:
            max_idx = max(range(len(arredondados)), key=lambda i: arredondados[i])
            arredondados[max_idx] += resto
        pesos_dict = dict(zip(pesos_dict.keys(), arredondados))

    for idx, (i, _) in enumerate(colunas_indices):
        cel_idx = (row[0].row, i + 1)
        if cel_idx in color_map:
            row[colunas_indices[idx][0]].value = None
            row[colunas_pesos[idx][0]].value = None
            row[colunas_fator[idx][0]].value = None

    usados = 0
    for idx, (i, _) in enumerate(colunas_indices):
        cel_idx = (row[0].row, i + 1)
        if cel_idx in color_map and usados < len(pesos_dict):
            indice = list(pesos_dict.keys())[usados]
            peso = round(pesos_dict[indice], 2)
            row[colunas_indices[idx][0]].value = indice
            row[colunas_pesos[idx][0]].value = peso
            row[colunas_fator[idx][0]].value = 1.0
            usados += 1

    formula_partes = []
    for k, v in pesos_dict.items():
        formula_partes.append(f"{round(v,2):.2f}.{k}")
    formula_sintetica = "+".join(formula_partes)
    ws.cell(row=row[0].row, column=col_formula).value = formula_sintetica
    formulas_set.add(formula_sintetica)

# Adicionar coluna com fórmulas únicas
col_unicas = col_formula + 1
ws.cell(row=1, column=col_unicas).value = "Formulas unicas"
formulas_unicas_ordenadas = list(formulas_set)

for i, formula in enumerate(formulas_unicas_ordenadas, start=2):
    ws.cell(row=i, column=col_unicas).value = formula

# Salvar novo arquivo
output_path = "C:\\Users\\FUDO\\Documents\\CSFR\\doc-saida\\csfr_com_formula_unica.xlsx"
wb.save(output_path)
print(f"Arquivo salvo em: {output_path}")

Arquivo salvo em: C:\Users\FUDO\Documents\CSFR\doc-saida\csfr_com_formula_unica.xlsx
